### Try-it 9.2: Predicting Wages

This activity is meant to summarize your work with regularized regression models.  You will use your earlier work with data preparation and pipelines together with what you've learned with grid searches to determine an optimal model.  In addition to the prior strategies, this example is an excellent opportunity to utilize the `TransformedTargetRegressor` estimator in scikitlearn.

### The Data

This dataset is loaded from the openml resource library.  Originally from census data, the data contains wage and demographic information on 534 individuals. From the dataset documentation [here](https://www.openml.org/d/534)

```
The Current Population Survey (CPS) is used to supplement census information between census years. These data consist of a random sample of 534 persons from the CPS, with information on wages and other characteristics of the workers, including sex, number of years of education, years of work experience, occupational status, region of residence and union membership. 
```

In [2]:
from sklearn.datasets import fetch_openml

In [3]:
wages = fetch_openml(data_id=534, as_frame=True)

In [5]:
wages.frame.head()

,EDUCATION,SOUTH,SEX,EXPERIENCE,UNION,WAGE,AGE,RACE,OCCUPATION,SECTOR,MARR
0,8,no,female,21,not_member,5.10,35,Hispanic,Other,Manufacturing,Married
1,9,no,female,42,not_member,4.95,57,White,Other,Manufacturing,Married
2,12,no,male,1,not_member,6.67,19,White,Other,Manufacturing,Unmarried
3,12,no,male,4,not_member,4.00,22,White,Other,Other,Unmarried
4,12,no,male,17,not_member,7.50,35,White,Other,Other,Married


In [6]:
df = wages.frame
features = df.columns.tolist()
features.remove('WAGE')

#### Task

Build regression models to predict `WAGE`.  Incorporate the categorical features and transform the target using a logarithm.  Build `Ridge` models and consider some different amounts of regularization.  

After fitting your model, interpret the model and try to understand what features led to higher wages.  Consider using `permutation_importance` that you encountered in module 8.  Discuss your findings in the class forum.

For an in depth example discussing the perils of interpreting the coefficients see the example in scikitlearn examples [here](https://scikit-learn.org/stable/auto_examples/inspection/plot_linear_model_coefficient_interpretation.html).

In [7]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.preprocessing import PolynomialFeatures, StandardScaler, OneHotEncoder
from sklearn.compose import make_column_transformer, TransformedTargetRegressor
from sklearn.inspection import permutation_importance
from sklearn.feature_selection import SequentialFeatureSelector
import numpy as np
import plotly.express as px
import pandas as pd
import warnings

In [32]:
# Set up training and test sets
trainX, testX, trainY, testY = train_test_split(df[features], df['WAGE'], test_size=0.3, random_state=1080)

In [33]:
# Define pipeline
pipeline = Pipeline([
  ('preprocess', make_column_transformer(
    (OneHotEncoder(drop='if_binary'), ['SOUTH','SEX','UNION','RACE','MARR','SECTOR','OCCUPATION']),
    (PolynomialFeatures(include_bias=False), ['EDUCATION','EXPERIENCE','AGE']))),
  ('scale', StandardScaler()),
  ('regressor', TransformedTargetRegressor(Ridge(), func=np.log, inverse_func=np.exp))
])

param_grid = {'regressor__regressor__alpha': [10**pp for pp in range(-1,4)],
              'preprocess__polynomialfeatures__degree': [1,2,3]}
gridSearch = GridSearchCV(pipeline, param_grid=param_grid, cv=10, scoring='neg_mean_squared_error')

In [34]:
gridSearch.fit(trainX, trainY)
cvResults = gridSearch.cv_results_

In [35]:
results = pd.DataFrame({
  'degree': [list(d.values())[0] for d in cvResults['params']],
  'alphaValues': [list(d.values())[1] for d in cvResults['params']],
  'Avg MSE': cvResults['mean_test_score']*-1
})

In [36]:
px.line(results, x='alphaValues', y='Avg MSE', facet_col='degree')

In [37]:
results.sort_values('Avg MSE')

,degree,alphaValues,Avg MSE
10,3,0.1,21.552696
5,2,0.1,21.584266
6,2,1.0,21.613338
11,3,1.0,21.651785
3,1,100.0,21.666327
2,1,10.0,21.681389
8,2,100.0,21.688478
12,3,10.0,21.717982
1,1,1.0,21.727191
0,1,0.1,21.734396


In [57]:
pd.DataFrame({
  'coefficient':gridSearch.best_estimator_.named_steps.preprocess.get_feature_names_out(),
  'value':gridSearch.best_estimator_.named_steps.regressor.regressor_.coef_
}).sort_values('value', key=abs, ascending=False)

,coefficient,value
19,polynomialfeatures__EDUCATION^2,1.049221
17,polynomialfeatures__EXPERIENCE,1.001582
22,polynomialfeatures__EXPERIENCE^2,-0.772186
24,polynomialfeatures__AGE^2,-0.574514
23,polynomialfeatures__EXPERIENCE AGE,-0.521732
18,polynomialfeatures__AGE,0.512116
25,polynomialfeatures__EDUCATION^3,-0.460812
21,polynomialfeatures__EDUCATION AGE,-0.385164
26,polynomialfeatures__EDUCATION^2 EXPERIENCE,0.355127
27,polynomialfeatures__EDUCATION^2 AGE,-0.349243


In [66]:
imps = permutation_importance(gridSearch, trainX, trainY,
                              n_repeats=30,
                              random_state=0, scoring='neg_mean_squared_error')

In [67]:
imps['importances_mean']

array([2.95133982e+01, 4.37211431e-01, 2.16267159e+00, 2.82531300e+03,
       6.80477795e-01, 7.73092349e+02, 1.92826055e-01, 2.67414522e+00,
       3.31093189e-01, 4.77368920e-02])

In [73]:
transTargReg = gridSearch.best_estimator_.named_steps.regressor

In [75]:
transTargReg.regressor_.

Ridge(alpha=0.1)